# Export a model


In this section of the lab you will export the trained model and learn how to use it in your application. 
Custom Vision Service allows classifiers to be exported to run offline. You can embed your exported classifier into an application and run it locally on a device for real-time classification.

Custom Vision Service supports the following exports:

- Tensorflow for Android.
- CoreML for iOS11.
- ONNX for Windows ML.
- A Windows or Linux container. The container includes a Tensorflow model and service code to use the Custom Vision Service API.

Custom Vision Service only exports compact domains. The models generated by compact domains are optimized for the constraints of real-time classification on low powered devices. Classifiers built with a compact domain may be slightly less accurate than a standard domain with the same amount of training data.



## Before you start
Install `wget` package that will be used later in the lab

In [ ]:
# Install Custom Vision Service SDK  in the current Jupyter kernel
import sys
!{sys.executable} -m pip install wget

## Retrain with a compact domain
If your classifier was not trained with the compact domain (which is the case for the classifier trained during the first stage of the lab) you need to retrain it.

In [ ]:
training_key = 'b1cbbf0f9a054ef481113c9efec1fe4d'
prediction_key = 'b01851d0bbe24e9ba3c542ce84306787'
project_name = 'AerialClassifier'

In [ ]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

trainer = training_api.TrainingApi(training_key)

### Change the project's domain

In [ ]:
# Print the list of available domains
for domain in trainer.get_domains():
 print(domain)

In [ ]:
# Find the ID of General (compact) domain
domain_name = 'General (compact)'
domain_id = None
for domain in trainer.get_domains():
    if domain.name == domain_name:
        domain_id = domain.id
        print("Found domain: {}  with ID: {}".format(domain_name,domain_id))
        break

if domain_id == None:
    print("Could not find domain: {}".format(domain_name))

In [ ]:
# Find you project's ID
project_id = None
project = None
for prj in trainer.get_projects():
    if prj.name == project_name:
        project_id = prj.id
        project = prj
        print("Found project: {0}".format(project_id))
        break
        
if project_id == None:
    print("Could not find your project")


In [ ]:
# Change the project's domain
project.settings.domain_id = domain_id
project = trainer.update_project(project_id, project)

### Retrain

In [ ]:
import time

def train(training_key, project_id):
    trainer = training_api.TrainingApi(training_key)
    print("Starting training...")
    try:
        iteration = trainer.train_project(project.id)
    except:
        print("No need to retrain")
        return
    
    while (iteration.status != "Completed"):
        iteration = trainer.get_iteration(project.id, iteration.id)
        print ("Training status: " + iteration.status)
        time.sleep(2)

    # The iteration is now trained. Make it the default project endpoint
    trainer.update_iteration(project_id, iteration.id, is_default=True)
    print("Done")
    return iteration.id

In [ ]:
# Start training
iteration_id = train(training_key, project_id)

## Export the iteration
Exporting a model is a two-step process. First you must request the export. It is an asynchronous call. You have to periodically check the status of the request and when the export package is ready you can download it using the returned URI.




### Request the export

In [ ]:
print("Requesting export for Iteration ID: {0}".format(iteration_id))

platform = 'TensorFlow'
flavor = 'Linux'

export = trainer.export_iteration(project_id, iteration_id, platform, flavor)
while (export.status != 'Done'):
   print ("Export status: " + export.status)
   time.sleep(2)
   export = trainer.get_exports(project_id, iteration_id)[0]

print("Export package ready. Download URI: {}", export.download_uri)    
    

### Download the package

In [ ]:
import wget
import os

download_filename = 'aerialclassifier.pb'

print("Downloading from: {0}".format(export.download_uri))
wget.download(export.download_uri, download_filename)

In [ ]:
%%sh
ls -l 